In [ ]:
%pylab inline

import sys
sys.path.insert(0, "../")

import numpy as np
from sklearn.externals import joblib
import xarray as xr
from xnoah.data_matrix import unstack_cat, stack_cat
import pandas as pd

In [ ]:
from lib.models import WeightedOutput, get_lrf
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

In [ ]:
X = xr.open_dataset("X.nc")#.pipe(mysel)
Y = xr.open_dataset("Y.nc")#.pipe(mysel)
w = xr.open_dataarray("w.nc")

# mu = X.mean(['time', 'x', 'y'])

In [ ]:
d = WeightedOutput.quickfit("X.nc", "Y.nc", "w.nc")
x, y, mod = map(d.get, ['x', 'y', 'mod'])
mod.score(x,y)

In [ ]:
lrf = get_lrf(mod, x, y)

plt.pcolormesh(lrf.loc["SL"]["Q1"].T)
plt.colorbar()

In [ ]:
plt.pcolormesh(lrf.loc["QT"]["Q1"].T)
plt.colorbar()

Why is this quantity so big and not smooth.

In [ ]:
lrf.loc["QT"].iloc[:,1].reset_index()

The height level of this point is 19800 m. The index corresponding to this height is 28. Let's look at what the data is like around there.

In [ ]:
def unprep(x):
    x_d = unstack_cat(x, "features").unstack("samples")
    # need to fix the fact that x_d['z'] has dtype object
    x_d['z'] = np.asarray(x_d['z'], dtype=float)
    return x_d

x_d = unprep(x)
y_d = unprep(y)

In [ ]:
x_d.isel(z=28, time=-21).QT.plot()

In [ ]:
x_d.QT.isel(time=-21, y=8).plot()

In [ ]:
qt = x_d.QT.mean(['x', 'y', 'time'])
semilogx(qt, qt.z)
plt.plot(qt[28], qt.z[28], '*', markersize=20)

My guess is that 19800m is the highest level at which $q_t$ is still dynamically influenced and correlated with convection in some way. Because it is the highest level, it is also least moist so the corresponding cofficients in the linear response function must be very large. I am not sure why Zhiming Kuang did not have this problem. Did he constrain his analysis to the troposphere?

Overall, this indicates that we needs a more effective way to normalize the data.